In [1]:
from gan_synthesis.u_net_models.contract import UNet

In [2]:
model = UNet()

In [ ]:
import torch
from gan_synthesis.preprocessing.transforms import read_cropped


image = read_cropped(t="contrast", idx=123, as_type="pt").unsqueeze(0)
model.eval()
with torch.no_grad():
    recon = model(image)

In [4]:
recon.shape

torch.Size([1, 128, 6, 6])